R magic
=============

In [170]:
# activate R magic
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [221]:
%cd /content/drive/My Drive/Colab Notebooks/AI_relative/Data_mining_in_R/tutorial/dataset/OLAP
!ls

/content/drive/My Drive/Colab Notebooks/AI_relative/Data_mining_in_R/tutorial/dataset/OLAP


Generate dimension tables and store in .csv
================================

In [229]:
%%R
cheese_table <- data.frame(key=1:3,
            name=c("Swiss", "Cheddar", "Mozzzarella"),
             supplier_key=1:3)
write.csv(cheese_table,
          "cheese_table.csv",
          row.names=F)
cheese_table

  key        name supplier_key
1   1       Swiss            1
2   2     Cheddar            2
3   3 Mozzzarella            3


In [230]:
%%R
dough_table <- data.frame(key=1:3,
            name=c("whole wheat thin", "white regular", "stuffed crust"),
             supplier_key=1:3)
write.csv(dough_table,
          "dough_table.csv",
          row.names=F)
dough_table

  key             name supplier_key
1   1 whole wheat thin            1
2   2    white regular            2
3   3    stuffed crust            3


In [231]:
%%R
size_table <- data.frame(key=1:4,
             name=c("small", "medium", "large", "xlarge"),
             specification=c("8 inches", "12 inches", "14 inches", "16 inches"))
write.csv(size_table,
          "size_table.csv",
          row.names=F)
size_table

  key   name specification
1   1  small      8 inches
2   2 medium     12 inches
3   3  large     14 inches
4   4 xlarge     16 inches


In [232]:
%%R
supplier_table <- data.frame(key=1:3,
             name=c("Ubisoft", "EA", "KONAMI"))
write.csv(supplier_table,
          "supplier_table.csv",
          row.names=F)
supplier_table

  key    name
1   1 Ubisoft
2   2      EA
3   3  KONAMI


In [233]:
%%R
city_table <- data.frame(key=1:4,
             name=c("Ottawa", "Toronto", "Edmonton", "Calgary"),
             province_key=1:4)
write.csv(city_table,
          "city_table.csv",
          row.names=F)
city_table

  key     name province_key
1   1   Ottawa            1
2   2  Toronto            2
3   3 Edmonton            3
4   4  Calgary            4


In [234]:
%%R
location_table <- data.frame(key=1:4,
             name=c("1123, Holland Street", "304, Baseline Street", "5463, Friday Street", "43, Somerset Street"),
             city_key=1:4)
write.csv(location_table,
          "location_table.csv",
          row.names=F)
location_table

  key                 name city_key
1   1 1123, Holland Street        1
2   2 304, Baseline Street        2
3   3  5463, Friday Street        3
4   4  43, Somerset Street        4


In [235]:
%%R
province_table <- data.frame(key=1:2,
             name=c("Ontario", "Alberta"),
             country=c("Canada", "Canada"))
write.csv(province_table,
          "province_table.csv",
          row.names=F)
province_table

  key    name country
1   1 Ontario  Canada
2   2 Alberta  Canada


In [236]:
%%R
store_table <- data.frame(key=1:4,
             location_key=1:4,
             manager=c("Mike", "Bob", "Mary", "Jessie"))
write.csv(store_table,
          "store_table.csv",
          row.names=F)
store_table

  key location_key manager
1   1            1    Mike
2   2            2     Bob
3   3            3    Mary
4   4            4  Jessie


In [237]:
%%R
topping_table <- data.frame(key=1:4,
             name=c("tomatoes", "pepper", "onions", "pepperoni"))
write.csv(topping_table,
          "topping_table.csv",
          row.names=F)
topping_table

  key      name
1   1  tomatoes
2   2    pepper
3   3    onions
4   4 pepperoni


In [238]:
%%R
month_table <- 
  data.frame(key=1:12,
            desc=c("Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"),
            quarter=c("Q1","Q1","Q1","Q2","Q2","Q2","Q3","Q3","Q3","Q4","Q4","Q4"))
write.csv(month_table,
          "month_table.csv",
          row.names=F)
month_table

   key desc quarter
1    1  Jan      Q1
2    2  Feb      Q1
3    3  Mar      Q1
4    4  Apr      Q2
5    5  May      Q2
6    6  Jun      Q2
7    7  Jul      Q3
8    8  Aug      Q3
9    9  Sep      Q3
10  10  Oct      Q4
11  11  Nov      Q4
12  12  Dec      Q4


generate fact table and store in .csv
============================

sample from all necessary dimension tables

In [181]:
%%R
sample_num = 100
store <- sample(store_table$key, sample_num, replace=T)
month <- sample(month_table$key, sample_num, replace=T)
cheese <- sample(cheese_table$key, sample_num, replace=T)
topping <- sample(topping_table$key, sample_num, replace=T)
size <- sample(size_table$key, sample_num, replace=T)
dough <- sample(dough_table$key, sample_num, replace=T)

the profit is determined by size, so we have

In [182]:
%%R
profit <- 1 * size

and we hope the amount of xlarge pizza will increase along with the increase of the month
$$ amount = month * size\_weights(size) + noise$$

In [195]:
%%R
size_weights <- function(x) {
   if(x <= 2){
       w <- 0
   } else if(x == 3){
       w <- 1
   }
   else{
       w <- 3
   }
   w
}

weight <- sapply(size, size_weights)
print(head(size, 20))
print(head(weight, 20))
noise <- sample(1:10, sample_num, replace=T)
amount <- month*weight+noise
print(head(amount, 20))

 [1] 4 4 2 4 3 2 4 1 3 3 1 3 2 1 4 1 4 1 3 3
 [1] 3 3 0 3 1 0 3 0 1 1 0 1 0 0 3 0 3 0 1 1
 [1] 35 46 10 10 15  2 20  6 15 15 10 12  5  6 45 10 25  8 11  9


we assemble them together to get a function

In [240]:
%%R
size_weights <- function(x) {
   if(x <= 2){
       w <- 0
   } else if(x == 3){
       w <- 1
   }
   else{
       w <- 3
   }
   w
}

generate_orders <- function(sample_num){
    store <- sample(store_table$key, sample_num, replace=T)
    month <- sample(month_table$key, sample_num, replace=T)
    cheese <- sample(cheese_table$key, sample_num, replace=T)
    topping <- sample(topping_table$key, sample_num, replace=T)
    size <- sample(size_table$key, sample_num, replace=T)
    dough <- sample(dough_table$key, sample_num, replace=T)

    profit <- 1 * size

    weight <- sapply(size, size_weights)
    noise <- sample(1:10, sample_num, replace=T)
    amount <- month*weight+noise

    # generate the fact table(order)
    order_table <- data.frame(store_key=store,
                      month_key=month,
                      size_key=size,
                      dough_key=dough,
                      cheese_key=cheese,
                      topping_key=topping,
                      amount=amount,
                      profit=profit)
    order_table <- order_table[order(order_table$store_key, -order_table$month_key),]
    row.names(order_table) <- NULL
    return(order_table)
}

new let's generate a fact table

In [244]:
%%R
order_table <- generate_orders(5000)
write.csv(order_table,
          "order_table.csv")
head(order_table,20)

   store_key month_key size_key dough_key cheese_key topping_key amount profit
1          1        12        1         1          3           3      9      1
2          1        12        3         1          2           3     19      3
3          1        12        1         1          2           1      8      1
4          1        12        3         1          2           1     17      3
5          1        12        4         3          2           4     43      4
6          1        12        4         3          1           3     46      4
7          1        12        3         3          1           4     17      3
8          1        12        1         3          2           3      3      1
9          1        12        3         3          1           3     20      3
10         1        12        4         3          2           2     46      4
11         1        12        2         2          2           4      4      2
12         1        12        3         1          1

read from .csv
==================================

In [247]:
%%R
month_table <- read.csv("month_table.csv")
topping_table <- read.csv("topping_table.csv")
store_table <- read.csv("store_table.csv")
province_table <- read.csv("province_table.csv")
location_table <- read.csv("location_table.csv")
city_table <- read.csv("city_table.csv")
supplier_table <- read.csv("supplier_table.csv")
size_table <- read.csv("size_table.csv")
dough_table <- read.csv("dough_table.csv")
cheese_table <- read.csv("cheese_table.csv")
order_table <- read.csv("order_table.csv")

OLAP operation
=========================

In [248]:
%%R
dimension <- c("store_key", "month_key", "size_key", "dough_key", "cheese_key", "topping_key")
revenue_cube <- 
    tapply(order_table$amount, 
           order_table[,dimension], 
           FUN=function(x){return(sum(x))})


show the cube of all dimension

In [243]:
%%R
revenue_cube

, , size_key = 1, dough_key = 1, cheese_key = 1, topping_key = 1

         month_key
store_key  1  2  3  4  5  6  7  8  9 10 11 12
        1 NA 10  1  7 NA 20  1 NA  6 NA  9 10
        2 12 10 NA 25 10  7 NA 10  8 NA 12  8
        3 NA 24  5  1 NA NA  9  6 NA NA  3  7
        4 NA NA NA  2 16  1 NA NA NA NA NA 12

, , size_key = 2, dough_key = 1, cheese_key = 1, topping_key = 1

         month_key
store_key  1  2  3  4  5  6  7  8  9 10 11 12
        1 NA  6  8 NA  8 NA NA 11 NA  2 20 12
        2 NA NA  4 NA  9 12 NA NA  8  6  6 NA
        3 10 NA NA NA NA  4 NA NA 13  2 NA 25
        4 NA NA NA NA NA  5  6 13 NA NA  1 NA

, , size_key = 3, dough_key = 1, cheese_key = 1, topping_key = 1

         month_key
store_key  1  2  3  4  5  6  7  8  9 10 11 12
        1 NA NA NA NA NA 21 NA 11 13 NA 35 15
        2  7 16  5 17 NA 12 NA 16 14 11 39 17
        3 NA NA 13 NA 11 NA NA 30 NA NA NA 14
        4  3 15  4 NA  8 11 NA 16 11 NA NA 61

, , size_key = 4, dough_key = 1, cheese_key = 1, top

check the dimension name of the cube

In [199]:
%%R
dimnames(revenue_cube)

$store_key
[1] "1" "2" "3" "4"

$month_key
 [1] "1"  "2"  "3"  "4"  "5"  "6"  "7"  "8"  "9"  "10" "11" "12"

$size_key
[1] "1" "2" "3" "4"

$dough_key
[1] "1" "2" "3"

$cheese_key
[1] "1" "2" "3"

$topping_key
[1] "1" "2" "3" "4"



roll up and drill down
===========================
month and size

In [200]:
%%R
apply(revenue_cube, c("month_key", "size_key"),
      FUN=function(x) {return(sum(x, na.rm=TRUE))})

         size_key
month_key   1   2    3    4
       1  604 593  811  906
       2  615 620  846 1407
       3  577 674  768 1480
       4  626 539 1149 1487
       5  588 540 1061 2312
       6  631 555  954 2184
       7  613 604 1332 2609
       8  598 508 1377 2823
       9  577 578 1487 3722
       10 587 569 1458 3556
       11 653 481 1529 3850
       12 611 506 1720 4279


month, size and store

In [201]:
%%R
apply(revenue_cube, c("month_key", "size_key", "store_key"),
      FUN=function(x) {return(sum(x, na.rm=TRUE))})

, , store_key = 1

         size_key
month_key   1   2   3    4
       1  119 170 172  235
       2  176 137 141  353
       3  114 206 227  268
       4  164 131 278  404
       5  152 108 282  557
       6  109 131 350  428
       7  141 122 301  668
       8  166 145 324  633
       9   99 139 350  811
       10 195 114 383  641
       11 147 127 514  649
       12 203 150 408 1002

, , store_key = 2

         size_key
month_key   1   2   3    4
       1  179 161 230  241
       2   92 151 289  304
       3  122 151 155  439
       4  149 112 307  278
       5  200 123 417  615
       6  161 119 186  649
       7  159 156 345  736
       8  152 148 315  803
       9  193 172 453  869
       10 164 192 361 1062
       11 214 115 398 1271
       12 169 126 551 1124

, , store_key = 3

         size_key
month_key   1   2   3    4
       1  142 150 218  246
       2  201 148 207  404
       3  177 146 193  370
       4  128 112 321  384
       5  115 157 150  496
       6  192 189 199  